In [1]:
# Import libraries
import pandas as pd
import geopandas as gpd
# Operations on geometries
import shapely
import numpy as np
from numpy import nan
import math
import glob 
import xarray as xr
import xarray
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
import tensorflow as tf
import statsmodels.api as sm
import xgboost
import xgboost as xgb
import csv as csv
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn import linear_model
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error,mean_squared_error
from sklearn.model_selection import TimeSeriesSplit,GridSearchCV
from sklearn.model_selection import cross_validate

from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor  #Perforing grid search
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn import datasets, metrics, model_selection
from yellowbrick.regressor import residuals_plot


import scipy
from scipy import stats
import os

import matplotlib.pyplot as plt

from scipy.stats import skew
from collections import OrderedDict

import _pickle as cPickle

import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

from utils_functions.utils_data import *
from utils_functions.utils_ml import *
from utils_functions.utils_RF import *
from utils_functions.utils_plot import *

/Users/noeliaotero/opt/anaconda3/envs/py39/lib/python3.9/site-packages/seaborn/apionly.py:9: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


In [9]:
PATH =  '/Users/noeliaotero/Documents/OCCR/data/Output_data/csv/outmodels_resevervoirs/simulations_PREVAH/Classical/models/'

In [10]:
PATH_CH2018 = '/Users/noeliaotero/Documents/OCCR/data/Output_data/csv/CH2018_tomodel_Level_reservoirs/'
PATH_OUT = '/Users/noeliaotero/Documents/OCCR/data/Output_data/csv/outmodels_resevervoirs/climate_rcp_simulations/RF_predsplit_CH2018/'
os.makedirs(PATH_OUT, exist_ok = True)

In [11]:
names_ch = os.listdir(PATH_CH2018)

In [12]:
nfiles = os.listdir(PATH)

In [45]:
rf_mod = []
names_mod = []
for i in range(0, len(nfiles)):
    ifile = ["rf_Predefinesplit" in i for i in nfiles] 
    if ifile[i]:
        #print(nfiles[i])
        with open(PATH + nfiles[i], 'rb') as fid:
            model = cPickle.load(fid)
        rf_mod.append(model)
        names_mod.append(nfiles[i].split("_")[0])

In [16]:
T1 = [2021,2061]
T2 = [2059, 2099]

In [29]:
colnames = ['date', 't2m','prec', 'prec7D', 'prec15D', 'prec30D', 't2m7D', 't2m15D',
       't2m30D','PREVAH','PREVAH7D', 'PREVAH15D', 'PREVAH30D','Week']
# Adding doi
Features = ['t2m','t2m2W','t2m4W', 't2m6W','precip', 'prec2W', 'prec4W',
       'prec6W','PREVAH','PREVAH2W', 'PREVAH4W', 'PREVAH6W', 'Week']

In [51]:
def get_CH2018_Reser_predictions (PATH_CH2018, rf_model, Features, PATH_OUT, mypipeline = None):
    """Get the predictions based on the RF model from the historical period
       Args: PATH_CH2018: directory with the files
             nam_mod: name of the model/station"""

 
    basenames = os.listdir(PATH_CH2018)
    name_ch = [os.path.splitext(x)[0] for x in basenames]

    
    x = readfiles(PATH_CH2018)
    m = rf_model
    l_predictions = []
    
    for i in range(0, len(name_ch)):
        print(i)
      
        #print(x[i].head())
        #if (x[i].name_p.unique()[0] != nam_mod):
        #    raise Exception('Something is not matching!')
        df = x[i]
        
        df_new = df
        df_new["date"] = pd.to_datetime(df_new["date"])
        # create the doi
        
        df_ref = df_new[(df_new.date.dt.year >= 1981) &  (df_new.date.dt.year <= 2021)]
        df_proj_ref = df_ref[Features]
        df_proj= df_new[Features]
        # scale the data
        if mypipeline == None:
            # select the same period 1981-2021 to scale
            #fpipeline = prepareData(df_proj)
            fpipeline = prepareData(df_proj_ref)
            X_proj = fpipeline.fit_transform(df_proj)
        elif mypipeline == 'FullRCP':
            fpipeline = prepareData(df_proj)
            X_proj = fpipeline.fit_transform(df_proj)    
        else:
            fpipeline = mypipeline
            X_proj = fpipeline.transform(df_proj)
            
        # Get the predictions
        predictions = m.model.predict(X_proj)
        # add the predictions to the data frame
        df_new['predictions'] = predictions
        # save the predictions
        PATH_stat = PATH_OUT + '/' 
        os.makedirs(PATH_stat, exist_ok=True)
        print('save predictions for', name_ch[i])
        df_new.to_csv(PATH_stat + name_ch[i] + '.csv')
        
    l_predictions.append(df_new)
        
    return(l_predictions)

In [52]:
get_CH2018_Reser_predictions(PATH_CH2018, rf_mod[0], Features, PATH_OUT, mypipeline = None)

0
save predictions for SMHI-RCA-ECEARTH-EUR11-RCP85
1
save predictions for DMI-HIRHAM-ECEARTH-EUR11-RCP26
2
save predictions for SMHI-RCA-ECEARTH-EUR11-RCP45
3
save predictions for SMHI-RCA-MPIESM-EUR11-RCP85
4
save predictions for DMI-HIRHAM-ECEARTH-EUR11-RCP85
5
save predictions for SMHI-RCA-MPIESM-EUR11-RCP45
6
save predictions for DMI-HIRHAM-ECEARTH-EUR11-RCP45
7
save predictions for SMHI-RCA-ECEARTH-EUR11-RCP26
8
save predictions for SMHI-RCA-HADGEM-EUR11-RCP45
9
save predictions for SMHI-RCA-HADGEM-EUR11-RCP85


[            date    PREVAH     precip        t2m    t2mmax  Week     prec2W  \
 0     1981-01-01  0.715592   0.000000  -9.200866 -1.693530     0   1.974873   
 1     1981-01-02  0.741041   3.949747  -1.415987  2.391529     0   1.994532   
 2     1981-01-03  0.790163   0.039317  -5.224739 -2.725140     0   0.019658   
 3     1981-01-04  0.740204   0.000000  -9.109478 -5.917553     0   0.000000   
 4     1981-01-05  0.695980   0.000000 -11.109892 -9.137886     1   0.000000   
 ...          ...       ...        ...        ...       ...   ...        ...   
 43459 2099-12-27  1.798306   0.020991  -3.810543 -0.855702    51   0.010496   
 43460 2099-12-28  1.732367   0.000000  -5.510808 -0.206201    52   0.001681   
 43461 2099-12-29  1.670898   0.003363  -0.827552  3.584162    52   0.428346   
 43462 2099-12-30  1.618796   0.853330  -0.022501  4.294568    52  17.297493   
 43463 2099-12-31  2.778694  33.741657   1.935253  5.552727    52  33.741657   
 
           prec4W     prec6W   t2max2W